In [ ]:
# main.py (excerpt) — unified, minimal, reproducible

import os, sys, json
sys.path.append("/home/ul/ul_student/ul_cep22/my_folders/drl_projekt")
import logging
logging.getLogger('mujoco').setLevel(logging.ERROR)

import gymnasium as gym, gymnasium_robotics, mujoco, numpy as np
from utils.change_parameters import (
    jprint, snapshot_geom_and_body,
    apply_override, apply_friction_override,
    contact_friction_product
)

os.environ['MUJOCO_GL'] = 'egl'
os.environ['PYOPENGL_PLATFORM'] = 'egl'

config_path = "/home/ul/ul_student/ul_cep22/my_folders/drl_projekt/configs/test.json"
config = json.load(open(config_path))

gym.register_envs(gymnasium_robotics)
env = gym.make(config["env_name"], render_mode="rgb_array")
env.reset()

# ===== BEFORE: compiled (XML-derived) state =====
snap_before = snapshot_geom_and_body(env, "object0")
jprint("[BEFORE  XML/COMPILED]", snap_before)

config["object_gid"]           = snap_before["geom"]["gid"]
config["object_type_initial"]  = snap_before["geom"]["type"]
config["object_size_initial"]  = snap_before["geom"]["size"]
config["body_mass_initial"]    = snap_before["body"]["mass"]
config["body_inertia_initial"] = snap_before["body"]["inertia_diag"]
config["friction_initial"]     = snap_before["geom"]["friction"]
config["condim_initial"]       = snap_before["geom"]["condim"]

ovr = config.get("override_object")
if ovr:
    jprint("[REQUESTED SHAPE OVERRIDE]", {
        "name": ovr.get("name", "object0"),
        "type": ovr.get("type"),
        "size": ovr.get("size"),
        "mass": ovr.get("mass", None),
        "update_inertia": bool(ovr.get("update_inertia", False)),
    })
    res_shape = apply_override(
        env,
        name=ovr.get("name", "object0"),
        typ=ovr["type"],
        size=ovr["size"],
        mass=ovr.get("mass", None),
        update_inertia=bool(ovr.get("update_inertia", False)),
        atol=1e-9,
    )
    jprint("[AFTER  SHAPE]", {"ok": res_shape["ok"], "after": res_shape["after"]})
    # record
    snap_shape = snapshot_geom_and_body(env, ovr.get("name", "object0"))
    config["object_type_final"]      = snap_shape["geom"]["type"]
    config["object_size_final"]      = snap_shape["geom"]["size"]
    config["body_mass_final"]        = snap_shape["body"]["mass"]
    config["body_inertia_final"]     = snap_shape["body"]["inertia_diag"]
    config["object_inertia_updated"] = res_shape["inertia_updated"]

# ===== FRICTION OVERRIDE =====
# "override_friction": {"name":"object0", "friction":[0.8, 0.0, 0.0], "condim":1}
fovr = config.get("override_friction")
if fovr:
    jprint("[REQUESTED FRICTION OVERRIDE]", fovr)
    res_fric = apply_friction_override(
        env,
        name=fovr.get("name", "object0"),
        friction=fovr["friction"],
        condim=fovr.get("condim", None),
        atol=1e-9,
    )
    jprint("[AFTER  FRICTION]", res_fric)
    config["friction_final"] = res_fric["after"]["friction"]
    config["condim_final"]   = res_fric["after"]["condim"]

    try:
        eff = contact_friction_product(env, fovr.get("name", "object0"), "geom_22")
        jprint("[CONTACT FRICTION object0 x table]", {"effective": eff.tolist()})
    except Exception:
        pass


[BEFORE  XML/COMPILED] {
  "geom": {
    "name": "object0",
    "gid": 23,
    "type": "box",
    "size": [
      0.025,
      0.025,
      0.025
    ],
    "dataid": -1,
    "friction": [
      1.0,
      0.005,
      0.0001
    ],
    "condim": 3
  },
  "body": {
    "bid": 32,
    "mass": 2.0,
    "inertia_diag": [
      0.0008333333333333335,
      0.0008333333333333335,
      0.0008333333333333335
    ]
  }
}
[REQUESTED SHAPE OVERRIDE] {
  "name": "object0",
  "type": "ellipsoid",
  "size": [
    0.025,
    0.02,
    0.015
  ],
  "mass": 2.0,
  "update_inertia": true
}
[AFTER  SHAPE] {
  "ok": true,
  "after": {
    "gid": 23,
    "type": "ellipsoid",
    "size": [
      0.025,
      0.02,
      0.015
    ],
    "dataid": -1,
    "body_id": 32,
    "friction": [
      1.0,
      0.005,
      0.0001
    ],
    "condim": 3
  }
}
[REQUESTED FRICTION OVERRIDE] {
  "name": "object0",
  "friction": [
    0.8,
    0.0,
    0.0
  ],
  "condim": 1
}
[AFTER  FRICTION] {
  "ok": true,
  "bef